# Boolean retrieval implementation and execution over TREC-DL 2020 Passage Ranking task

In [1]:
import nltk
import string

import re

import numpy as np

import time

from scipy.sparse import csr_matrix, lil_matrix
from multiprocessing import Pool

import pickle
import gc

import os

import scipy.sparse as scipy_sparse

from datetime import datetime

In [2]:
MSMARCO_PASSAGE_COLLECTION="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/pyserini/collections/msmarco-passage/collection.tsv"

STEMMED_DOCS_FILE="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/ia368v_dd_class_02/indexes/tempfile.pkl"

STEMMED_DOCS_FILE_FORMAT="{}_{}.pkl"
STEMMED_DOCS_FOLDER="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/ia368v_dd_class_02/indexes"

REVERSED_INDEX_FOLDER="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/ia368v_dd_class_02/indexes"
REVERSED_INDEX_FILE_FORMAT="reversed_index_TREC-DL_2020_{}.pkl"

REVERSED_INDEX_FILE="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/ia368v_dd_class_02/indexes/reversed_index_TREC-DL_2020_complete.pkl"

TREC_DL_2020_QUERIES="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/pyserini/collections/trec-dl_2020-passage/msmarco-test2020-queries.tsv"
TREC_DL_2020_QRELS="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/pyserini/collections/trec-dl_2020-passage/2020qrels-pass.txt"

TREC_DL_2020_RUN_FORMAT="TREC_DL_2020_boolean_run_{}.tsv"
TREC_DL_2020_RUN_FOLDER="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/ia368v_dd_class_02/runs"

TREC_EVAL_FULLPATH="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval"

NUMBER_OF_DOCS=8841822

PARTIAL_STEMMED_DOCS_COUNT=1000000

## Text preprocessing functions

In [3]:
stop_words = set(nltk.corpus.stopwords.words('english'))
punctuation = set(string.punctuation)
stemmer = nltk.stem.PorterStemmer()

In [4]:
def test_preprocess_text(which_text):

    all_tokens = nltk.word_tokenize(which_text.lower())
    cleaned_tokens = [token for token in all_tokens if token not in stop_words and token not in punctuation]
    stemmed_tokens = [stemmer.stem(token) for token in cleaned_tokens]

    return stemmed_tokens

## Read all the documents and their corresponding indexes

In [ ]:
read_documents_text = []
read_documents_index = []

In [ ]:
with open(MSMARCO_PASSAGE_COLLECTION, 'r', encoding="utf-8") as inputFile:
    
    for i in range(NUMBER_OF_DOCS):
        line = inputFile.readline()
        cleaned = re.sub("(â.{1}.{1})", " ", line)
        document_data = cleaned.split('\t')        

        read_documents_text.append(document_data[1])
        read_documents_index.append(document_data[0])

In [ ]:
len(read_documents_text)

In [ ]:
read_documents_text[715779]

In [ ]:
read_documents_index[1038342]

## Now, preprocess all the documents in parallel

In [ ]:
with Pool(processes=8) as pool:
    stemmed_documents = pool.map(test_preprocess_text, read_documents_text)

In [ ]:
with open(STEMMED_DOCS_FILE, "wb") as outputFile:
    pickle.dump({"doc_ids": read_documents_index, 
                 "stemmed_docs": stemmed_documents}, outputFile, pickle.HIGHEST_PROTOCOL)

In [ ]:
del read_documents_text

In [ ]:
del reversed_index

In [ ]:
gc.collect()

In [ ]:
len(stemmed_documents)

### Split the stemmed document file in smaller ones

In [ ]:
with open(STEMMED_DOCS_FILE, "rb") as inputFile:
    
    temp_file = pickle.load(inputFile)
    
    read_documents_index = temp_file['doc_ids']
    stemmed_documents = temp_file['stemmed_docs']

In [ ]:
for i in range(NUMBER_OF_DOCS // PARTIAL_STEMMED_DOCS_COUNT + 1):
    with open(os.path.join(STEMMED_DOCS_FOLDER, STEMMED_DOCS_FILE_FORMAT.format("temp", i)), "wb") as outputFile:
        pickle.dump({"doc_ids": read_documents_index[i * PARTIAL_STEMMED_DOCS_COUNT:(i * PARTIAL_STEMMED_DOCS_COUNT) + PARTIAL_STEMMED_DOCS_COUNT], 
                     "stemmed_docs": stemmed_documents[i * PARTIAL_STEMMED_DOCS_COUNT:(i * PARTIAL_STEMMED_DOCS_COUNT) + PARTIAL_STEMMED_DOCS_COUNT]}, outputFile, pickle.HIGHEST_PROTOCOL)

In [ ]:
del read_documents_index
del stemmed_documents
del temp_file

In [ ]:
gc.collect()

## Finally, compute the reversed index using a sparse matrix to store the documents' word frequencies

In [ ]:
#
# This function loads the partial file with the stemmed documents and computes the reversed index for the terms in the
# documents.
#
# After that, the reversed index computed to the part of the entire collection is saved to be processed later.
#

def process_stemmed_docs_file(file_index):
    
    reversed_index = {}
    
    with open(os.path.join(STEMMED_DOCS_FOLDER, STEMMED_DOCS_FILE_FORMAT.format("temp", file_index)), "rb") as inputFile:
        temp_file = pickle.load(inputFile)

        read_documents_index = temp_file['doc_ids']
        stemmed_documents = temp_file['stemmed_docs']                

    print("Document range: {} until {}".format(read_documents_index[0], read_documents_index[-1]))
        
    start_time = time.time()
        
    for doc_index in range(len(stemmed_documents)):
        
        if doc_index % 100000 == 0:
            print(time.time() - start_time)
            print(doc_index)
            start_time = time.time()
        
        for token in stemmed_documents[doc_index]:
            if token not in reversed_index:
                reversed_index[token] = lil_matrix((1, NUMBER_OF_DOCS), dtype=np.short)

            reversed_index[token][0, int(read_documents_index[doc_index])] += 1

    del read_documents_index
    del stemmed_documents
    del temp_file            
            
    with open(os.path.join(REVERSED_INDEX_FOLDER, REVERSED_INDEX_FILE_FORMAT.format(file_index)), 'wb') as outputFile:
        pickle.dump(reversed_index, outputFile, pickle.HIGHEST_PROTOCOL)
    
    return True

### Process in parallel all the stemmed documents file parts

In [ ]:
with Pool(processes=2) as pool:
    results = pool.map(process_stemmed_docs_file, range(0, NUMBER_OF_DOCS // PARTIAL_STEMMED_DOCS_COUNT + 1))

In [ ]:
gc.collect()

### Load the precomputed reversed indexes and consolidate them into a single dictionary

The reversed index file had to be created and saved in chunks due to local RAM limitations. However, it is possible to consolidate it into a single dictionary and save it later.

In [ ]:
start_time = time.time()

with open(os.path.join(REVERSED_INDEX_FOLDER, REVERSED_INDEX_FILE_FORMAT.format(0)), 'rb') as inputFile:
    reversed_indexes = pickle.load(inputFile)

for i in range(1, NUMBER_OF_DOCS // PARTIAL_STEMMED_DOCS_COUNT + 1):
    with open(os.path.join(REVERSED_INDEX_FOLDER, REVERSED_INDEX_FILE_FORMAT.format(i)), 'rb') as inputFile:
        reversed_indexes_tmp = pickle.load(inputFile)
        
    for token, token_docs in reversed_indexes_tmp.items():
        if token in reversed_indexes:
            reversed_indexes[token] = reversed_indexes[token] + token_docs
        else:
            reversed_indexes[token] = token_docs
            
print(time.time() - start_time)

In [ ]:
gc.collect()

In [ ]:
len(reversed_indexes)

In [ ]:
with open(REVERSED_INDEX_FILE, 'wb') as outputFile:
    pickle.dump(reversed_indexes, outputFile, pickle.HIGHEST_PROTOCOL)

## .. or read the preprocessed data

In [5]:
with open(REVERSED_INDEX_FILE, 'rb') as inputFile:
    reversed_indexes = pickle.load(inputFile)

In [6]:
len(reversed_indexes)

3750155

## Now run the queries

In [7]:
query_index = []
query_text = []

### Read all the queries in memory

In [8]:
with open(TREC_DL_2020_QUERIES, 'r', encoding="utf-8") as inputFile:
    for line in inputFile:
        query_data = line.split('\t')
        
        query_index.append(query_data[0])
        query_text.append(query_data[1])

In [9]:
len(query_text)

200

### Preprocess the queries the same way as the documents

In [10]:
with Pool(processes=8) as pool:
    stemmed_queries = pool.map(test_preprocess_text, query_text)

In [11]:
len(stemmed_queries)

200

### Find the query matches

In [12]:
def find_related_documents(stemmed_query, only_all_terms=False):
    
    related_documents = {}
    
    for token in stemmed_query:
        if token in reversed_indexes:
            
#             print(token)
            
            related_docs = reversed_indexes[token].nonzero()
            
            if type(reversed_indexes[token]) is scipy_sparse._csr.csr_matrix:
                related_docs_scores = np.array(reversed_indexes[token][related_docs])[0]
            else:
                related_docs_scores = np.array(reversed_indexes[token][related_docs].todense())[0]
            
            for i, doc in enumerate(related_docs[1]):
                if only_all_terms:
                    if not related_documents:
                        related_documents[doc] = 0

                    if doc in related_documents:
                        related_documents[doc] += related_docs_scores[i]
                else:
                    if doc not in related_documents:
                        related_documents[doc] = 0
                    
                    related_documents[doc] += related_docs_scores[i]
            
    return related_documents

### Compute the queries considering valid all documents which matches at least one query token/term

In [14]:
process_start_time = time.time()

queries_matches = []

for i in range(200):
    start_time = time.time()

    print("{} : {}".format(i, stemmed_queries[i]))
    
    queries_matches.append(find_related_documents(stemmed_queries[i]))

    print("{}\n".format(time.time() - start_time))
    
print("Total time to process the queries: {}\n".format(time.time() - process_start_time))    

0 : ['aziz', 'hashim']
0.0010097026824951172

1 : ['rep', 'scalis']
0.00784611701965332

2 : ['kill', 'nichola', 'ii', 'russia']
0.1593766212463379

3 : ['own', 'barnhart', 'crane']
0.052989959716796875

4 : ['said', 'one', 'make', 'feel', 'inferior']
2.7456607818603516

5 : ['sing', 'monk', 'theme', 'song']
0.12708401679992676

6 : ['highest', 'career', 'passer', 'rate', 'nfl']
0.564629077911377

7 : ['hunter', 'pattern', 'shotgun']
0.07120704650878906

8 : ['place', 'scalp', 'feel', 'sore']
0.680077314376831

9 : ['pete', 'rose', 'ban', 'hall', 'fame']
0.08832788467407227

10 : ['thoma', 'cooley']
0.027951478958129883

11 : ['definit', 'endors']
0.3172156810760498

12 : ['hormon', 'increas', 'calcium', 'level', 'blood']
1.0190961360931396

13 : ['defin', 'geon']
0.14927005767822266

14 : ['amazon', 'rainforest', 'locat']
0.4899137020111084

15 : ['four', 'forc', 'act', 'airplan', 'equilibrium']
0.5460882186889648

16 : ['defin', 'pareto', 'chart', 'statist']
0.28179454803466797

17 :

0.49726366996765137

135 : ['averag', 'wed', 'dress', 'alter', 'cost']
0.8669941425323486

136 : ['project', 'definit']
0.41779232025146484

137 : ['barclay', 'fca', 'number']
0.5434868335723877

138 : ['benefit', 'polici', 'layoff']
0.2692375183105469

139 : ['hour', 'clinic']
0.43642401695251465

140 : ['symptom', 'shingl']
0.2636117935180664

141 : ['biggest', 'loser', 'challeng']
0.08940529823303223

142 : ['villag', 'burnham']
0.035643577575683594

143 : ['vitamin', 'e', 'anti', 'scar']
0.14042162895202637

144 : ['weather', 'antigua', 'novemb']
0.18820977210998535

145 : ['weather', 'novi', 'sad']
0.13579607009887695

146 : ['best', 'food', 'lower', 'cholesterol']
0.9675536155700684

147 : ['carvedilol', 'use']
1.9784164428710938

148 : ['caus', 'bruis', 'appear']
0.8336241245269775

149 : ['caus', 'muscl', 'tear']
0.7905693054199219

150 : ['counti', 'dexter', 'michigan']
0.25754284858703613

151 : ['counti', 'new', 'york', 'new', 'york']
1.322805643081665

152 : ['counti', 'rio

### Now save the results in the TREC format

In [15]:
TREC_RESULT_LINE_FORMAT="{}\tQ0\t{}\t{}\t{}\tboolean\n"

In [16]:
MAX_RESULTS_TO_SAVE=1000

In [17]:
def generate_trec_format(queries_matches, query_ids, output_filename, verbose=False):
    
    with open(output_filename, 'w') as outputFile:
        for i, query_result in enumerate(queries_matches):
            
            if verbose:
                print("Saving query {}\n{}:".format(i, query_text[i]))
            
            relevant_docs = np.array(list(query_result.keys()))
            relevant_docs_scores = np.array(list(query_result.values()))
            
            relevant_docs_order = np.argsort(relevant_docs_scores)[::-1]
            
            if verbose:
                print("relevant_docs.shape={}".format(relevant_docs.shape))
            
            relevant_docs_final_result = relevant_docs[relevant_docs_order]
            relevant_docs_final_score = relevant_docs_scores[relevant_docs_order]
            
            if verbose:
                print("relevant_docs_final_result: {}\n\n".format(relevant_docs_final_result))
            
            for j, each_match in enumerate(relevant_docs_final_result[:MAX_RESULTS_TO_SAVE]):
                outputFile.write(TREC_RESULT_LINE_FORMAT.format(query_ids[i], each_match, j, relevant_docs_final_score[j]))

In [18]:
run_filename = os.path.join(TREC_DL_2020_RUN_FOLDER, TREC_DL_2020_RUN_FORMAT.format(datetime.now().strftime("%Y%m%d_%H%M%S")))

In [19]:
generate_trec_format(queries_matches, query_index, run_filename)

### Now, apply the TREC metrics

In [20]:
!{TREC_EVAL_FULLPATH} -c -mrecall.1000 -mmap -mndcg_cut.10 -mrecip_rank \
    {TREC_DL_2020_QRELS} {run_filename}

map                   	all	0.0259
recip_rank            	all	0.1238
recall_1000           	all	0.2519
ndcg_cut_10           	all	0.0538


### Now, compute the queries considering valid only documents which matches all query tokens/terms

In [21]:
process_start_time = time.time()

queries_matches = []

for i in range(200):
    start_time = time.time()

    print("{} : {}".format(i, stemmed_queries[i]))
    
    queries_matches.append(find_related_documents(stemmed_queries[i], only_all_terms=True))

    print("{}\n".format(time.time() - start_time))
    
print("Total time to process the queries: {}\n".format(time.time() - process_start_time))    

0 : ['aziz', 'hashim']
0.0007412433624267578

1 : ['rep', 'scalis']
0.0017991065979003906

2 : ['kill', 'nichola', 'ii', 'russia']
0.023960351943969727

3 : ['own', 'barnhart', 'crane']
0.005112648010253906

4 : ['said', 'one', 'make', 'feel', 'inferior']
0.2449817657470703

5 : ['sing', 'monk', 'theme', 'song']
0.011463403701782227

6 : ['highest', 'career', 'passer', 'rate', 'nfl']
0.04585433006286621

7 : ['hunter', 'pattern', 'shotgun']
0.006354808807373047

8 : ['place', 'scalp', 'feel', 'sore']
0.05582237243652344

9 : ['pete', 'rose', 'ban', 'hall', 'fame']
0.008560657501220703

10 : ['thoma', 'cooley']
0.0027692317962646484

11 : ['definit', 'endors']
0.026799917221069336

12 : ['hormon', 'increas', 'calcium', 'level', 'blood']
0.09621715545654297

13 : ['defin', 'geon']
0.01302337646484375

14 : ['amazon', 'rainforest', 'locat']
0.040873050689697266

15 : ['four', 'forc', 'act', 'airplan', 'equilibrium']
0.0465235710144043

16 : ['defin', 'pareto', 'chart', 'statist']
0.024923

0.1803731918334961

134 : ['oracl', 'bind', 'variabl', 'exampl']
0.044457435607910156

135 : ['averag', 'wed', 'dress', 'alter', 'cost']
0.08219027519226074

136 : ['project', 'definit']
0.03685927391052246

137 : ['barclay', 'fca', 'number']
0.04958200454711914

138 : ['benefit', 'polici', 'layoff']
0.026459217071533203

139 : ['hour', 'clinic']
0.04004788398742676

140 : ['symptom', 'shingl']
0.023311376571655273

141 : ['biggest', 'loser', 'challeng']
0.008419275283813477

142 : ['villag', 'burnham']
0.003537416458129883

143 : ['vitamin', 'e', 'anti', 'scar']
0.012970924377441406

144 : ['weather', 'antigua', 'novemb']
0.017078876495361328

145 : ['weather', 'novi', 'sad']
0.012034177780151367

146 : ['best', 'food', 'lower', 'cholesterol']
0.08992958068847656

147 : ['carvedilol', 'use']
0.1778888702392578

148 : ['caus', 'bruis', 'appear']
0.07793927192687988

149 : ['caus', 'muscl', 'tear']
0.07368040084838867

150 : ['counti', 'dexter', 'michigan']
0.023970603942871094

151 : [

### Save the results in the TREC format, indicating the "all terms" variation

In [22]:
run_filename = os.path.join(TREC_DL_2020_RUN_FOLDER, TREC_DL_2020_RUN_FORMAT.format("all_terms_" + datetime.now().strftime("%Y%m%d_%H%M%S")))

In [23]:
generate_trec_format(queries_matches, query_index, run_filename)

### Now, apply the TREC metrics

In [24]:
!{TREC_EVAL_FULLPATH} -c -mrecall.1000 -mmap -mndcg_cut.10 -mrecip_rank \
    {TREC_DL_2020_QRELS} {run_filename}

map                   	all	0.0004
recip_rank            	all	0.0185
recall_1000           	all	0.0004
ndcg_cut_10           	all	0.0017
